In [221]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from prophet import Prophet

from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import TimeSeriesSplit

import os
os.chdir('..')
import gc
import sys


import datetime
from utils import utils, models

In [3]:
daily_df = pd.read_csv('data/daily_data.csv')

In [17]:
daily_df.tail()

,station,date,temp_max,temp_mean,temp_min,rainfall,snow
76667,PALH0,2023-11-18,-6.7,-10.191667,-13.3,False,False
76668,PALH0,2023-11-19,-10.6,-13.812500,-16.1,False,False
76669,PALH0,2023-11-20,-10.6,-16.162500,-18.9,False,False
76670,PALH0,2023-11-21,-4.4,-9.512500,-12.8,False,False
76671,PALH0,2023-11-22,-3.9,-4.911765,-6.4,True,True


In [18]:
# test_date = str(datetime.date.today() - datetime.timedelta(days=4))
test_date = '2023-11-16'
daily = True

In [20]:
daily_df["date"] = pd.to_datetime(daily_df["date"])


In [21]:
train_data_pd, test_data_pd = models.train_test_split(daily_df, test_date, daily)

In [22]:
train_data_pd.tail()

,station,date,temp_max,temp_mean,temp_min,rainfall,snow
76660,PALH0,2023-11-11,-0.5,-3.529167,-7.2,True,True
76661,PALH0,2023-11-12,-2.5,-6.104167,-10.0,False,False
76662,PALH0,2023-11-13,-2.2,-4.766667,-6.1,True,True
76663,PALH0,2023-11-14,-3.0,-6.720833,-10.0,False,False
76664,PALH0,2023-11-15,2.2,-3.370833,-6.0,True,True


In [23]:
test_data_pd

,station,date,temp_max,temp_mean,temp_min,rainfall,snow
3646,72202,2023-11-16,27.0,23.775000,21.0,True,False
3647,72202,2023-11-17,28.3,25.033333,23.0,True,False
3648,72202,2023-11-18,26.1,23.812500,22.0,True,False
3649,72202,2023-11-19,29.0,24.333333,21.0,False,False
3650,72202,2023-11-20,29.4,25.304167,20.6,False,False
...,...,...,...,...,...,...,...
76667,PALH0,2023-11-18,-6.7,-10.191667,-13.3,False,False
76668,PALH0,2023-11-19,-10.6,-13.812500,-16.1,False,False
76669,PALH0,2023-11-20,-10.6,-16.162500,-18.9,False,False
76670,PALH0,2023-11-21,-4.4,-9.512500,-12.8,False,False


# Prophet training

### First for one station 

In [56]:
train_1 = train_data_pd.loc[train_data_pd.station == '72202']
test_1 = test_data_pd.loc[test_data_pd.station == '72202']

In [57]:
train_1 = pd.DataFrame(train_1.rename(columns={'date': 'ds', 'temp_min': 'y'}))
test_1 = pd.DataFrame(test_1.rename(columns={'date': 'ds', 'temp_min': 'y'}))

### Adding regressors

In [26]:
model = Prophet()

In [27]:
train_1.columns

Index(['station', 'ds', 'temp_max', 'temp_mean', 'y', 'rainfall', 'snow'], dtype='object')

In [28]:
model.add_regressor(name='temp_max')
model.add_regressor(name='temp_mean')
model.add_regressor(name='rainfall')
model.add_regressor(name='snow')

In [29]:
model.fit(train_1[['ds', 'y', 'temp_max', 'temp_mean', 'rainfall', 'snow']])

10:27:27 - cmdstanpy - INFO - Chain [1] start processing
10:27:27 - cmdstanpy - INFO - Chain [1] done processing


In [31]:
future = model.make_future_dataframe(periods=7)
future = future.merge(test_1[['ds', 'temp_max', 'temp_mean', 'rainfall', 'snow']], on='ds') 

In [32]:
forecast_1 = model.predict(future)

In [33]:
eval = (pd.DataFrame(test_1[['ds', 'y']])
              .merge(forecast_1[['ds', 'yhat']], on='ds')
              )

In [35]:
eval.head()

,ds,y,yhat
0,2023-11-16,21.0,20.864130
1,2023-11-17,23.0,22.177245
2,2023-11-18,22.0,21.341177
3,2023-11-19,21.0,20.522439
4,2023-11-20,20.6,21.985756


In [37]:
np.sqrt(mean_squared_error(eval.y, eval.yhat))

0.7565998817695939

#### Avg temp

In [58]:
train_1 = pd.DataFrame(train_1.rename(columns={'date': 'ds', 'y': 'temp_min'})).rename(columns={'temp_mean': 'y'})
test_1 = pd.DataFrame(test_1.rename(columns={'date': 'ds', 'y': 'temp_min'})).rename(columns={'temp_mean': 'y'})

In [62]:
model = Prophet()
model.add_regressor(name='temp_max')
model.add_regressor(name='temp_min')
model.add_regressor(name='rainfall')
model.add_regressor(name='snow')

model.fit(train_1[['ds', 'y', 'temp_max', 'temp_min', 'rainfall', 'snow']])

future = model.make_future_dataframe(periods=7)
future = future.merge(test_1[['ds', 'temp_max', 'temp_min', 'rainfall', 'snow']], on='ds')
forecast_1 = model.predict(future)


10:38:52 - cmdstanpy - INFO - Chain [1] start processing
10:38:53 - cmdstanpy - INFO - Chain [1] done processing


In [63]:
eval = (pd.DataFrame(test_1[['ds', 'y']])
              .merge(forecast_1[['ds', 'yhat']], on='ds')
              )

eval.head()
np.sqrt(mean_squared_error(eval.y, eval.yhat))

0.376564034237821

In [64]:
eval.head()

,ds,y,yhat
0,2023-11-16,23.775000,23.860366
1,2023-11-17,25.033333,25.458914
2,2023-11-18,23.812500,24.021066
3,2023-11-19,24.333333,24.755379
4,2023-11-20,25.304167,24.657423


In [65]:
train_1 = pd.DataFrame(train_1.rename(columns={'date': 'ds', 'y': 'temp_mean'})).rename(columns={'temp_max': 'y'})
test_1 = pd.DataFrame(test_1.rename(columns={'date': 'ds', 'y': 'temp_mean'})).rename(columns={'temp_max': 'y'})

In [66]:
model = Prophet()
model.add_regressor(name='temp_mean')
model.add_regressor(name='temp_min')
model.add_regressor(name='rainfall')
model.add_regressor(name='snow')

model.fit(train_1[['ds', 'y', 'temp_mean', 'temp_min', 'rainfall', 'snow']])

future = model.make_future_dataframe(periods=7)
future = future.merge(test_1[['ds', 'temp_mean', 'temp_min', 'rainfall', 'snow']], on='ds')

forecast_1 = model.predict(future)

eval = (test_1
        .merge(forecast_1[['ds', 'yhat', 'yhat_lower','yhat_upper']], on='ds')
              )

np.sqrt(mean_squared_error(eval.y, eval.yhat))

10:39:44 - cmdstanpy - INFO - Chain [1] start processing
10:39:45 - cmdstanpy - INFO - Chain [1] done processing


0.5742560326199894

In [67]:
eval

,station,ds,y,temp_mean,temp_min,rainfall,snow,yhat,yhat_lower,yhat_upper
0,72202,2023-11-16,27.0,23.775000,21.0,True,False,27.123190,26.009474,28.065607
1,72202,2023-11-17,28.3,25.033333,23.0,True,False,27.816811,26.668746,28.825116
2,72202,2023-11-18,26.1,23.812500,22.0,True,False,26.547128,25.420279,27.591037
3,72202,2023-11-19,29.0,24.333333,21.0,False,False,28.039737,26.975185,29.071069
4,72202,2023-11-20,29.4,25.304167,20.6,False,False,29.809738,28.747841,30.854994
5,72202,2023-11-21,28.3,26.345833,24.4,False,False,29.033973,27.979343,30.055078
6,72202,2023-11-22,28.9,25.500000,23.3,False,False,28.419257,27.475474,29.479997


In [68]:
train_1 = pd.DataFrame(train_1.rename(columns={'date': 'ds', 'y': 'temp_max'})).rename(columns={'snow': 'y'})
test_1 = pd.DataFrame(test_1.rename(columns={'date': 'ds', 'y': 'temp_max'})).rename(columns={'snow': 'y'})


model = Prophet()
model.add_regressor(name='temp_mean')
model.add_regressor(name='temp_min')
model.add_regressor(name='rainfall')
model.add_regressor(name='temp_max')

model.fit(train_1[['ds', 'y', 'temp_mean', 'temp_min', 'rainfall', 'temp_max']])

future = model.make_future_dataframe(periods=7)
future = future.merge(test_1[['ds', 'temp_mean', 'temp_min', 'rainfall', 'temp_max']], on='ds')
forecast_1 = model.predict(future)
eval = (test_1
        .merge(forecast_1[['ds', 'yhat', 'yhat_lower','yhat_upper']], on='ds')
              )

np.sqrt(mean_squared_error(eval.y, eval.yhat))

0.0

## Fit a model on all stations?

In [70]:
train_data_pd.head()

,station,date,temp_max,temp_mean,temp_min,rainfall,snow
0,72202,2013-11-22,26.7,25.957143,24.4,True,False
1,72202,2013-11-23,27.2,25.033333,23.9,True,False
2,72202,2013-11-24,28.3,24.620833,21.7,False,False
3,72202,2013-11-25,25.6,23.179167,21.7,True,False
4,72202,2013-11-26,27.2,25.016667,22.8,True,False


In [72]:
train_df = pd.get_dummies(train_data_pd, columns=['station'])
test_df = pd.get_dummies(test_data_pd, columns=['station'])

In [78]:
train_df = pd.DataFrame(train_df.rename(columns={'date': 'ds', 'temp_min' : 'y'}))
test_df = pd.DataFrame(test_df.rename(columns={'date': 'ds',  'temp_min': 'y'}))

In [73]:
model = Prophet()
model.add_regressor(name='temp_mean')
model.add_regressor(name='snow')
model.add_regressor(name='rainfall')
model.add_regressor(name='temp_max')

In [75]:
for col in test_df.columns:
    if 'station' in col:
        model.add_regressor(name=col)

In [80]:

model.fit(train_df)

10:48:43 - cmdstanpy - INFO - Chain [1] start processing
10:49:36 - cmdstanpy - INFO - Chain [1] done processing


In [82]:
future = model.make_future_dataframe(periods=7)
future = future.merge(test_df.drop('y', axis=1), on='ds')

forecast_1 = model.predict(future)

In [83]:
eval_model = (test_df
        .merge(forecast_1[['ds', 'yhat']], on='ds')
              )

In [88]:
np.sqrt(mean_squared_error(eval_model.y, eval_model.yhat))

11.80841528875813

In [89]:
station_ids = daily_df.station.unique()

In [92]:
stations = dict(zip(range(len(station_ids)), station_ids))

In [96]:
models_min = dict()

In [102]:
def model_predictor(y: str, X: list, train: pd.DataFrame, 
                    test: pd.DataFrame, stations: dict, days: int = 7):
    
    models = dict()
    for i, station in stations.items():
        train_1 = train.loc[train.station == station]
        test_1 = test.loc[test.station == station]

        train_1 = pd.DataFrame(train_1.rename(columns={'date': 'ds', y: 'y'}))
        test_1 = pd.DataFrame(test_1.rename(columns={'date': 'ds', y: 'y'}))

        models[i] = Prophet()
        for x in X:
            models[i].add_regressor(name=x)

        models[i].fit(train_1[['ds', 'y'] + X])

        future = models[i].make_future_dataframe(periods=days)
        future = future.merge(test_1[['ds'] + X], on='ds') 

        forecast_1 = models[i].predict(future)

        eval_m = (pd.DataFrame(test_1[['ds', 'y']])
                    .merge(forecast_1[['ds', 'yhat']], on='ds')
                    )

        print(f"MSE = {np.sqrt(mean_squared_error(eval_m.y, eval_m.yhat))}")


    return models

In [103]:
model_min = model_predictor('temp_min', 
                            ['temp_max', 'temp_mean', 'rainfall', 'snow'], 
                            train_data_pd, test_data_pd, stations)

11:06:22 - cmdstanpy - INFO - Chain [1] start processing
11:06:22 - cmdstanpy - INFO - Chain [1] done processing
11:06:22 - cmdstanpy - INFO - Chain [1] start processing
11:06:23 - cmdstanpy - INFO - Chain [1] done processing
11:06:23 - cmdstanpy - INFO - Chain [1] start processing
11:06:23 - cmdstanpy - INFO - Chain [1] done processing
11:06:24 - cmdstanpy - INFO - Chain [1] start processing
11:06:25 - cmdstanpy - INFO - Chain [1] done processing
11:06:25 - cmdstanpy - INFO - Chain [1] start processing
11:06:25 - cmdstanpy - INFO - Chain [1] done processing
11:06:26 - cmdstanpy - INFO - Chain [1] start processing
11:06:26 - cmdstanpy - INFO - Chain [1] done processing
11:06:26 - cmdstanpy - INFO - Chain [1] start processing
11:06:27 - cmdstanpy - INFO - Chain [1] done processing
11:06:27 - cmdstanpy - INFO - Chain [1] start processing
11:06:28 - cmdstanpy - INFO - Chain [1] done processing
11:06:28 - cmdstanpy - INFO - Chain [1] start processing
11:06:28 - cmdstanpy - INFO - Chain [1]

MSE = 0.7022818047205439


In [97]:
for i, station in stations.items():
    train_1 = train_data_pd.loc[train_data_pd.station == station]
    test_1 = test_data_pd.loc[test_data_pd.station == station]

    train_1 = pd.DataFrame(train_1.rename(columns={'date': 'ds', 'temp_min': 'y'}))
    test_1 = pd.DataFrame(test_1.rename(columns={'date': 'ds', 'temp_min': 'y'}))

    models_min[i] = Prophet()
    models_min[i].add_regressor(name='temp_max')
    models_min[i].add_regressor(name='temp_mean')
    models_min[i].add_regressor(name='rainfall')
    models_min[i].add_regressor(name='snow')

    models_min[i].fit(train_1[['ds', 'y', 'temp_max', 'temp_mean', 'rainfall', 'snow']])

    future = models_min[i].make_future_dataframe(periods=7)
    future = future.merge(test_1[['ds', 'temp_max', 'temp_mean', 'rainfall', 'snow']], on='ds') 

    forecast_1 = models_min[i].predict(future)

    eval_m = (pd.DataFrame(test_1[['ds', 'y']])
                .merge(forecast_1[['ds', 'yhat']], on='ds')
                )
                        
    models_min[station] = np.sqrt(mean_squared_error(eval_m.y, eval_m.yhat))

10:57:38 - cmdstanpy - INFO - Chain [1] start processing
10:57:38 - cmdstanpy - INFO - Chain [1] done processing
10:57:39 - cmdstanpy - INFO - Chain [1] start processing
10:57:39 - cmdstanpy - INFO - Chain [1] done processing
10:57:39 - cmdstanpy - INFO - Chain [1] start processing
10:57:40 - cmdstanpy - INFO - Chain [1] done processing
10:57:40 - cmdstanpy - INFO - Chain [1] start processing
10:57:41 - cmdstanpy - INFO - Chain [1] done processing
10:57:41 - cmdstanpy - INFO - Chain [1] start processing
10:57:41 - cmdstanpy - INFO - Chain [1] done processing
10:57:42 - cmdstanpy - INFO - Chain [1] start processing
10:57:42 - cmdstanpy - INFO - Chain [1] done processing
10:57:42 - cmdstanpy - INFO - Chain [1] start processing
10:57:43 - cmdstanpy - INFO - Chain [1] done processing
10:57:43 - cmdstanpy - INFO - Chain [1] start processing
10:57:44 - cmdstanpy - INFO - Chain [1] done processing
10:57:44 - cmdstanpy - INFO - Chain [1] start processing
10:57:44 - cmdstanpy - INFO - Chain [1]

# Xgboost

In [121]:
def xgb_features(df, label=None):
    """
    Creates time series features
    """

    if not daily_df.date.dtype == 'datetime64[ns]' :
        df['date'] = pd.to_datetime(df['date'])
    
    
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    
    X = df[['month','year',
           'dayofyear','dayofmonth']]
    if label:
        y = df[label]
        X = df.drop(label, axis=1)
        return X, y
    return df

In [122]:
xgb_data = xgb_features(daily_df)

In [126]:
xgb_data  = xgb_data.sort_values(by='date', ascending=True)

In [134]:
xgb_data['station'] = pd.Categorical(xgb_data['station'])

In [149]:
def dict_configs(d):
    for vcomb in itertools.product(*d.values()):
        yield dict(zip(d.keys(), vcomb))

In [184]:
xgb_data.drop([y,'date'], axis=1).columns

Index(['station', 'temp_max', 'temp_mean', 'rainfall', 'snow', 'month', 'year',
       'dayofyear', 'dayofmonth'],
      dtype='object')

In [199]:
xgb_data = pd.get_dummies(xgb_data, columns=['station'])

In [252]:
tss = TimeSeriesSplit(n_splits=3, test_size=4)


param_grid = {
    'max_depth': np.arange(2, 10, 2),
    'n_estimators': np.arange(100, 1200, 300),
}

params =  {'learning_rate': 0.1,
    'objective': 'reg:squarederror',
    'booster': 'gbtree',
    'n_jobs': 4,
    # 'enable_categorical': True,
    'random_state': 42}

fold = 0
# preds = []
scores = []
y = "temp_min"
xgb_data.dropna(inplace=True)
features = xgb_data.drop([y,'date'], axis=1).columns
grid = dict_configs(param_grid)
for train_idx, val_idx in tss.split(xgb_data):
    train = xgb_data.iloc[train_idx]
    test = xgb_data.iloc[val_idx]
    cv_results = dict()
    for cv_params in grid:
        X_train = train[features]
        y_train = train[y].shift(1).dropna()

        X_train.drop(index=X_train.index[:1], axis=0, inplace=True)

        X_test = test[features]
        y_test = test[y]

        params.update(cv_params)
        print(params)
        # reg = MultiOutputRegressor(xgb.XGBRegressor(**params))
        reg = xgb.XGBRegressor(**params)
        reg.fit(X_train, y_train,
                eval_set=[(X_train, y_train), (X_test, y_test)],
                verbose=0)

        y_pred = reg.predict(X_test)
#       preds.append(y_pred)
        cv_results[str(cv_params)] = np.sqrt(mean_squared_error(y_test, y_pred))
        scores.append(cv_results)

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


{'learning_rate': 0.1, 'objective': 'reg:squarederror', 'booster': 'gbtree', 'n_jobs': 4, 'random_state': 42, 'max_depth': 2, 'n_estimators': 100}


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


{'learning_rate': 0.1, 'objective': 'reg:squarederror', 'booster': 'gbtree', 'n_jobs': 4, 'random_state': 42, 'max_depth': 2, 'n_estimators': 400}


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...

{'learning_rate': 0.1, 'objective': 'reg:squarederror', 'booster': 'gbtree', 'n_jobs': 4, 'random_state': 42, 'max_depth': 8, 'n_estimators': 400}


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


{'learning_rate': 0.1, 'objective': 'reg:squarederror', 'booster': 'gbtree', 'n_jobs': 4, 'random_state': 42, 'max_depth': 8, 'n_estimators': 700}


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


{'learning_rate': 0.1, 'objective': 'reg:squarederror', 'booster': 'gbtree', 'n_jobs': 4, 'random_state': 42, 'max_depth': 8, 'n_estimators': 1000}


In [253]:
scores

[{"{'max_depth': 2, 'n_estimators': 100}": 5.742300992609579,
  "{'max_depth': 2, 'n_estimators': 400}": 5.8441100751865855,
  "{'max_depth': 2, 'n_estimators': 700}": 5.952860256466153,
  "{'max_depth': 2, 'n_estimators': 1000}": 6.006600847203467,
  "{'max_depth': 4, 'n_estimators': 100}": 5.533827505887801,
  "{'max_depth': 4, 'n_estimators': 400}": 6.708383722512493,
  "{'max_depth': 4, 'n_estimators': 700}": 6.541898164130732,
  "{'max_depth': 4, 'n_estimators': 1000}": 7.425928657166456,
  "{'max_depth': 6, 'n_estimators': 100}": 6.032389297751595,
  "{'max_depth': 6, 'n_estimators': 400}": 5.901004638846375,
  "{'max_depth': 6, 'n_estimators': 700}": 6.236974064477442,
  "{'max_depth': 6, 'n_estimators': 1000}": 6.657361783456716,
  "{'max_depth': 8, 'n_estimators': 100}": 6.926409826153647,
  "{'max_depth': 8, 'n_estimators': 400}": 6.665698654660439,
  "{'max_depth': 8, 'n_estimators': 700}": 7.325206339724756,
  "{'max_depth': 8, 'n_estimators': 1000}": 7.298569744355412},
 {

In [239]:
y_train.to_frame().assign(
    day1 = y_train.shift(1),
    day2 = y_train.shift(2),
    day3= y_train.shift(3),
    day4 = y_train.shift(4)
).drop(y, axis=1).dropna()


,day1,day2,day3,day4
36490,14.4,8.9,3.3,24.4
43796,1.1,14.4,8.9,3.3
29224,-1.7,1.1,14.4,8.9
51102,1.1,-1.7,1.1,14.4
25571,2.8,1.1,-1.7,1.1
...,...,...,...,...
32876,7.6,-2.0,7.1,23.3
10958,1.1,7.6,-2.0,7.1
7305,12.3,1.1,7.6,-2.0
43795,6.7,12.3,1.1,7.6


In [251]:
tss = TimeSeriesSplit(n_splits=3, test_size=4)

param_grid = {
    'max_depth': np.arange(2, 10, 2),
    'n_estimators': np.arange(100, 1200, 300),
}

params =  {'learning_rate': 0.1,
    'objective': 'reg:squarederror',
    'booster': 'gbtree',
    'n_jobs': 4,
    # 'enable_categorical': True,
    'random_state': 42}

fold = 0
# preds = []
scores = []
y = "temp_min"
xgb_data.dropna(inplace=True)
features = xgb_data.drop([y,'date'], axis=1).columns
grid = dict_configs(param_grid)
for train_idx, val_idx in tss.split(xgb_data):
    train = xgb_data.iloc[train_idx]
    test = xgb_data.iloc[val_idx]
    cv_results = dict()
    for cv_params in grid:
        X_train = train[features]
        y_train = train[y]

        y_train.to_frame().assign(
                day1 = y_train.shift(1),
                day2 = y_train.shift(2),
                day3= y_train.shift(3),
                day4 = y_train.shift(4)
            ).drop(y, axis=1).dropna()
        
        X_train.drop(index=X_train.index[:4], axis=0, inplace=True)

        X_test = test[features]
        y_test = test[y]
        y_test = y_test.to_numpy().reshape(-1, 4)

        params.update(cv_params)
        print(params)
        reg = xgb.XGBRegressor(**params)
        reg.fit(X_train, y_train,
                eval_set=[(X_train, y_train), (X_test, y_test)],
                verbose=0)

        y_pred = reg.predict(X_test)
#       preds.append(y_pred)
        cv_results[str(cv_params)] = np.sqrt(mean_squared_error(y_test, y_pred))
        scores.append(cv_results)

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


{'learning_rate': 0.1, 'objective': 'reg:squarederror', 'booster': 'gbtree', 'n_jobs': 4, 'random_state': 42, 'max_depth': 2, 'n_estimators': 100}


XGBoostError: [15:11:21] /home/conda/feedstock_root/build_artifacts/xgboost-split_1700181168148/work/src/data/data.cc:455: Check failed: this->labels.Size() % this->num_row_ == 0 (4 vs. 0) : Incorrect size for labels.
Stack trace:
  [bt] (0) /home/marc/miniconda3/envs/604Final/lib/libxgboost.so(+0xb6361) [0x7f8b52ab6361]
  [bt] (1) /home/marc/miniconda3/envs/604Final/lib/libxgboost.so(xgboost::MetaInfo::SetInfoFromHost(xgboost::GenericParameter const&, xgboost::StringView, xgboost::Json)+0x2ac2) [0x7f8b52bb32c2]
  [bt] (2) /home/marc/miniconda3/envs/604Final/lib/libxgboost.so(xgboost::MetaInfo::SetInfo(xgboost::GenericParameter const&, xgboost::StringView, xgboost::StringView)+0x80) [0x7f8b52bb3490]
  [bt] (3) /home/marc/miniconda3/envs/604Final/lib/libxgboost.so(XGDMatrixSetInfoFromInterface+0xec) [0x7f8b52ab8f4c]
  [bt] (4) /home/marc/miniconda3/envs/604Final/lib/python3.10/lib-dynload/../../libffi.so.8(+0x6a4a) [0x7f8be2e36a4a]
  [bt] (5) /home/marc/miniconda3/envs/604Final/lib/pyth

#### Classificaition

In [ ]:
tss = TimeSeriesSplit(n_splits=3, test_size=4)


param_grid = {
    'max_depth': np.arange(2, 10, 2),
    'n_estimators': np.arange(100, 1200, 300),
}

params =  {'learning_rate': 0.1,
    'objective': 'reg:logistic',
    'booster': 'gbtree',
    'n_jobs': 4,
    # 'enable_categorical': True,
    'random_state': 42}

fold = 0
# preds = []
scores = []
y = "snow"
xgb_data.dropna(inplace=True)
features = xgb_data.drop([y,'date'], axis=1).columns
grid = dict_configs(param_grid)
for train_idx, val_idx in tss.split(xgb_data):
    train = xgb_data.iloc[train_idx]
    test = xgb_data.iloc[val_idx]
    cv_results = dict()
    for cv_params in grid:
        X_train = train[features]
        y_train = train[y].shift(1).dropna()

        X_train.drop(index=X_train.index[:1], axis=0, inplace=True)

        X_test = test[features]
        y_test = test[y]

        params.update(cv_params)
        print(params)
        # reg = MultiOutputRegressor(xgb.XGBRegressor(**params))
        reg = xgb.XGBRegressor(**params)
        reg.fit(X_train, y_train,
                eval_set=[(X_train, y_train), (X_test, y_test)],
                verbose=0)

        y_pred = reg.predict(X_test)
#       preds.append(y_pred)
        cv_results[str(cv_params)] = np.sqrt(mean_squared_error(y_test, y_pred))
        scores.append(cv_results)

# XGB Lagged Feature


In [ ]:
xgb_data["temp_mean"].ewm(alpha=.9, adjust=False).mean().shape

# XGB Restrict Months
#### only include months November and December 